## [1] File count and transfer volume based on the Grafana

In [1]:
%%bash
# Grafana Parameters
grafana="https://monit-grafana.cern.ch/api/datasources/proxy/8035/query?db=monit_production_transfer" 
field_keys="src_experiment_site,count,transferred_volume,avg_operation_time,avg_file_size"
rp="one_month"
experiment="transfer_xrootd"

# Day Parameters and Input Parameters
HOUR_Z=$(date -u +%H)
echo INFO UTC HOUR NOW: $HOUR_Z
HOUR_W=10
YMD=2019-04-05
YMD_1DAY=$(date -d "1 day $YMD" +%Y-%m-%d)
input_root=xmfar-2019-04-partial.root
wanted=$YMD
TIME=${YMD_1DAY}T${HOUR_W}

wanted_day=$(date -d "$wanted" +%d) ; today=$(date +%d)
days_ago=$(expr $today - $wanted_day)

pastfuture="ago" ; [ $days_ago -lt 0 ] && pastfuture="" ; Ymd=$(date -d "$days_ago days $pastfuture" +%Y-%m-%d) ; wanted_day_plus_1=$(date -d "1 day $Ymd" +%d)
days_ago_plus_1=$(expr $today - $wanted_day_plus_1)

# Site
s=T2_US_Florida

# Grafana search condition
condition="vo='cms' AND remote_access =~ /^true/  AND technology = 'xrootd' AND time >= now() - ${days_ago}d and time <= now() - ${days_ago_plus_1}d AND src_experiment_site = '$s'"
echo INFO UTC HOUR NOW: $HOUR_Z Site=$s Wanted Date=$wanted Search Tim=$TIME

volume=0.0
countsum=0
grafana_number=$(curl -G -H "Authorization: Bearer $(cat token.txt)" "$grafana" --data-urlencode \
  "q=SELECT $field_keys from ${rp}.${experiment} WHERE $condition" 2>/dev/null | \
  sed "s#\[\"#\n\[\"#g" | \
  grep "\[\"[0-9][0-9][0-9][0-9]" | grep $TIME | \
  while read line ; do 
     time=$(echo $line | cut -d\" -f2)
     transferred_volume=$(echo $line | cut -d, -f4)
     count=$(echo $line | cut -d, -f3)
     volume=$(echo "scale=2 ; $volume + $transferred_volume" | bc )
     countsum=$(echo "scale=2 ; $countsum + $count" | bc)
     echo $time $volume $(echo "scale=2 ; $volume / 1024 / 1024" | bc)"MB" $countsum $transferred_volume
   done | tail -1)
   
echo INFO grafana_numbers on $(echo $TIME | cut -dT -f1) from $HOUR_W:00:00 $HOUR_W:59:59
echo INFO "File Count": $(echo $grafana_number | awk '{print $(NF-1)}')
#TranferVol=$(echo $grafana_number | awk '{print $(NF-2)}' | sed 's#MB##')
TranferVol=$(echo $grafana_number | awk '{print $(NF-2)}' | sed 's#MB##')
echo INFO "Transf Vol": $(echo "scale=2 ; $TranferVol / 1024 " | bc)"GB"


INFO UTC HOUR NOW: 16
INFO UTC HOUR NOW: 16 Site=T2_US_Florida Wanted Date=2019-04-05 Search Tim=2019-04-06T10
INFO grafana_numbers on 2019-04-06 from 10:00:00 10:59:59
INFO File Count: 3186
INFO Transf Vol: 1481.62GB


## [2] File count based on the root file at UCSD collector:
root://gftp-1.t2.ucsd.edu:1094//store/user/matevz/xmfar-2019-04-partial.root

In [1]:
%%bash
# Day Parameters and Input Parameters
HOUR_Z=$(date -u +%H)
echo INFO UTC HOUR NOW: $HOUR_Z
HOUR_W=10
YMD=2019-04-05
YMD_1DAY=$(date -d "1 day $YMD" +%Y-%m-%d)
input_root=xmfar-2019-04-partial.root
wanted=$YMD
TIME=${YMD_1DAY}T${HOUR_W}

wanted_day=$(date -d "$wanted" +%d) ; today=$(date +%d)
days_ago=$(expr $today - $wanted_day)

pastfuture="ago" ; [ $days_ago -lt 0 ] && pastfuture="" ; Ymd=$(date -d "$days_ago days $pastfuture" +%Y-%m-%d) ; wanted_day_plus_1=$(date -d "1 day $Ymd" +%d)
days_ago_plus_1=$(expr $today - $wanted_day_plus_1)

# Site
s=T2_US_Florida

cat << 'EOF' > readxmfarROOT.py
import os
import sys
import ROOT
import string
from datetime import datetime
    
root_file="xmfar-2019-04-partial.root"
root_file=str(sys.argv[1])
#YMD=str(sys.argv[2])
#YMD_1DAY=str(sys.argv[3])
TIME=str(sys.argv[2])
tfile = ROOT.TFile.Open(root_file)
tree = tfile.Get("XrdFar")
count=0
rBsum=0.0
for entry in tree:
    sN=str(getattr(entry,'S.mSite'))
    if 'T2_US_Florida' not in sN : continue
    rB=entry.GetLeaf('F.mRTotalMB').GetValue()
    wB=entry.GetLeaf('F.mWTotalMB').GetValue()
    t1=entry.GetLeaf('F.mOpenTime').GetValue()
    t2=entry.GetLeaf('F.mCloseTime').GetValue()
    sB=entry.GetLeaf('F.mSizeMB').GetValue()
    ap=str(getattr(entry,'U.mAppInfo'))
    lF=str(getattr(entry,'F.mName'))
    rN=str(getattr(entry,'U.mRealName'))
    closetime=str(datetime.utcfromtimestamp(t2).strftime('%Y-%m-%dT%H:%M:%SZ'))
    if TIME not in closetime: continue
    count=count+1
    #print datetime.utcfromtimestamp(t2).strftime('%Y-%m-%dT%H:%M:%SZ')," ","[",i,"]"," ",t2," ",t1," ",rB," ",wB," ",sB," ",sN," ",lF,rN,ap
    rBsum=rBsum+rB
print rBsum," ",count
EOF
rBsum_count=$(python readxmfarROOT.py $input_root $TIME | tail -1)
echo INFO xrd_numbers on $(echo $TIME | cut -dT -f1) from $HOUR_W:00:00 $HOUR_W:59:59
echo INFO rBsum_count : $rBsum_count
echo INFO "File Count": $(echo $rBsum_count | awk '{print $NF}')
TranferVol=$(echo $rBsum_count | awk '{print $(NF-1)}')
echo INFO "Transf Vol": $(echo "scale=2 ; $TranferVol / 1024 " | bc)"GB"


INFO UTC HOUR NOW: 16
INFO xrd_numbers on 2019-04-06 from 10:00:00 10:59:59
INFO rBsum_count : 1537273.33482 5143
INFO File Count: 5143
INFO Transf Vol: 1501.24GB


TClass::Init:0: RuntimeWarning: no dictionary for class SXrdFileInfo is available
TClass::Init:0: RuntimeWarning: no dictionary for class SRange is available
TClass::Init:0: RuntimeWarning: no dictionary for class SXrdUserInfo is available
TClass::Init:0: RuntimeWarning: no dictionary for class SXrdServerInfo is available


## Xrdmon and Grafana Transfer Volume Difference in %

In [6]:
%%bash
echo Xrdmon and Grafana Transfer Volume Difference in % : $(echo "scale=2 ; (1501.24 -  1481.62 ) / 1501.24 * 100 " | bc) %

Xrdmon and Grafana Transfer Volume Difference in % : 1.00 %


## Xrdmon and Grafana File Count Difference 

In [5]:
%%bash
echo Xrdmon and Grafana File Count Difference : $( expr 5143 - 3186 ) files

Xrdmon and Grafana File Count Difference : 1957 files
